In [56]:
import pandas as pd
import re
import seaborn as sns

In [87]:
# Loading the dataset
df = pd.read_csv("healthcare_dataset.csv")

In [88]:
# Normalizing the 0NF hospital names
df['Hospital'] = df['Hospital'].astype(str).str.strip().str.title()
df['Hospital_cleaned'] = df['Hospital'].str.replace(r',\s*', '|', regex=True)
df['Hospital_cleaned'] = df['Hospital_cleaned'].str.replace(r'\band\b', '|', regex=True, flags=re.IGNORECASE)
df['Hospital_cleaned'] = df['Hospital_cleaned'].str.replace(r'\|+', '|', regex=True)
df1NF = df.assign(Hospital=df['Hospital_cleaned'].str.split(r'\|')).explode('Hospital')
df1NF['Hospital'] = df1NF['Hospital'].str.strip()
df1NF = df1NF[df1NF['Hospital'].str.lower() != 'and']
df1NF = df1NF[df1NF['Hospital'] != '']

# Spliting Patinet Name' into First and Last
df1NF[['First_Name', 'Last_Name']] = df1NF['Name'].str.strip().str.split(' ', 1, expand=True)
# Spliting 'Doctor' into First and Last
df1NF[['Doctor_First_Name', 'Doctor_Last_Name']] = df1NF['Doctor'].str.strip().str.split(' ', 1, expand=True)

In [89]:
df1NF = df1NF[['PatientID', 'First_Name', 'Last_Name', 'Age', 'Gender', 'Blood Type', 'Medical Condition',
       'Date of Admission', 'Doctor_First_Name', 'Doctor_Last_Name', 'Hospital', 'Insurance Provider',
       'Billing Amount', 'Room Number', 'Admission Type', 'Discharge Date', 'Medication', 'Test Results']]

In [109]:
df1NF.head()
# df1NF = pd.read_csv("final_hospital_column_1NF.csv")

,PatientID,First_Name,Last_Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor_First_Name,Doctor_Last_Name,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,101,Bobby,JacksOn,30,Male,B-,Cancer,1/31/2024,Matthew,Smith,Sons,Blue Cross,18856.28131,328,Urgent,2/2/2024,Paracetamol,Normal
0,101,Bobby,JacksOn,30,Male,B-,Cancer,1/31/2024,Matthew,Smith,Miller,Blue Cross,18856.28131,328,Urgent,2/2/2024,Paracetamol,Normal
1,102,LesLie,TErRy,62,Male,A+,Obesity,8/20/2019,Samantha,Davies,Kim Inc,Medicare,33643.32729,265,Emergency,8/26/2019,Ibuprofen,Inconclusive
2,103,DaNnY,sMitH,76,Female,A-,Obesity,9/22/2022,Tiffany,Mitchell,Cook Plc,Aetna,27955.09608,205,Emergency,10/7/2022,Aspirin,Normal
3,104,andrEw,waTtS,28,Female,O+,Diabetes,11/18/2020,Kevin,Wells,Hernandez Rogers,Medicare,37909.78241,450,Elective,12/18/2020,Ibuprofen,Abnormal


In [110]:
df1NF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75027 entries, 0 to 55499
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   PatientID           75027 non-null  int64  
 1   First_Name          75027 non-null  object 
 2   Last_Name           75027 non-null  object 
 3   Age                 75027 non-null  int64  
 4   Gender              75027 non-null  object 
 5   Blood Type          75027 non-null  object 
 6   Medical Condition   75027 non-null  object 
 7   Date of Admission   75027 non-null  object 
 8   Doctor_First_Name   75027 non-null  object 
 9   Doctor_Last_Name    75027 non-null  object 
 10  Hospital            75027 non-null  object 
 11  Insurance Provider  75027 non-null  object 
 12  Billing Amount      75027 non-null  float64
 13  Room Number         75027 non-null  int64  
 14  Admission Type      75027 non-null  object 
 15  Discharge Date      75027 non-null  object 
 16  Medi

In [111]:
patient_df = df1NF[['PatientID', 'First_Name', 'Last_Name', 'Age', 'Gender', 'Blood Type', 'Medical Condition']].drop_duplicates()
patient_df.to_csv("patient_3NF.csv", index=False)
patient_df.head()

,PatientID,First_Name,Last_Name,Age,Gender,Blood Type,Medical Condition
0,101,Bobby,JacksOn,30,Male,B-,Cancer
1,102,LesLie,TErRy,62,Male,A+,Obesity
2,103,DaNnY,sMitH,76,Female,A-,Obesity
3,104,andrEw,waTtS,28,Female,O+,Diabetes
4,105,adrIENNE,bEll,43,Female,AB+,Cancer


In [112]:
patient_df.shape

(55500, 7)

In [113]:
hospital_df = df1NF[['Hospital']].drop_duplicates().reset_index(drop=True)
hospital_df['HospitalID'] = 200001 + hospital_df.index
hospital_df = hospital_df[['HospitalID', 'Hospital']]
hospital_df.to_csv("hospital_3NF.csv", index=False)
hospital_df.head()

,HospitalID,Hospital
0,200001,Sons
1,200002,Miller
2,200003,Kim Inc
3,200004,Cook Plc
4,200005,Hernandez Rogers


In [114]:
hospital_df.shape

(36751, 2)

In [128]:
doctor_df = df1NF[['Doctor_First_Name', 'Doctor_Last_Name']].drop_duplicates().reset_index(drop=True)
doctor_df['DoctorID'] = 300001 + doctor_df.index
doctor_df = doctor_df[['DoctorID', 'Doctor_First_Name', 'Doctor_Last_Name']]
doctor_df.to_csv("doctor_3NF.csv", index=False)
doctor_df.head()

,DoctorID,Doctor_First_Name,Doctor_Last_Name
0,300001,Matthew,Smith
1,300002,Samantha,Davies
2,300003,Tiffany,Mitchell
3,300004,Kevin,Wells
4,300005,Kathleen,Hanna


In [117]:
doctor_df.shape

(40341, 3)

In [118]:
insurance_df = df1NF[['Insurance Provider']].drop_duplicates().reset_index(drop=True)
insurance_df['InsuranceID'] = 'I' + (insurance_df.index + 1).astype(str)
insurance_df = insurance_df[['InsuranceID', 'Insurance Provider']]
insurance_df.to_csv("insurance_3NF.csv", index=False)
insurance_df

,InsuranceID,Insurance Provider
0,I1,Blue Cross
1,I2,Medicare
2,I3,Aetna
3,I4,UnitedHealthcare
4,I5,Cigna


In [119]:
insurance_df.shape

(5, 2)

In [120]:
medication_df = df1NF[['Medication']].drop_duplicates().reset_index(drop=True)
medication_df['MedicationID'] = 'M' + (medication_df.index + 1).astype(str)
medication_df = medication_df[['MedicationID', 'Medication']]
medication_df.to_csv("medication_3NF.csv", index=False)
medication_df.head()

,MedicationID,Medication
0,M1,Paracetamol
1,M2,Ibuprofen
2,M3,Aspirin
3,M4,Penicillin
4,M5,Lipitor


In [121]:
medication_df.shape

(5, 2)

In [122]:
patient_hospital_df = df1NF[['PatientID', 'Hospital', 'Date of Admission', 'Discharge Date', 'Room Number', 'Admission Type', 'Billing Amount']].drop_duplicates()
patient_hospital_df = patient_hospital_df.merge(hospital_df, on='Hospital', how='left')
patient_hospital_df = patient_hospital_df[['PatientID', 'HospitalID', 'Date of Admission', 'Discharge Date', 'Room Number', 'Admission Type', 'Billing Amount']]
patient_hospital_df.to_csv("patient_hospital_3NF.csv", index=False)
patient_hospital_df.head()

,PatientID,HospitalID,Date of Admission,Discharge Date,Room Number,Admission Type,Billing Amount
0,101,200001,1/31/2024,2/2/2024,328,Urgent,18856.28131
1,101,200002,1/31/2024,2/2/2024,328,Urgent,18856.28131
2,102,200003,8/20/2019,8/26/2019,265,Emergency,33643.32729
3,103,200004,9/22/2022,10/7/2022,205,Emergency,27955.09608
4,104,200005,11/18/2020,12/18/2020,450,Elective,37909.78241


In [123]:
patient_hospital_df.shape

(74990, 7)

In [124]:
patient_doctor_df = df1NF[['PatientID', 'Doctor_First_Name', 'Doctor_Last_Name']].drop_duplicates()
patient_doctor_df = patient_doctor_df.merge(doctor_df, on=['Doctor_First_Name', 'Doctor_Last_Name'], how='left')
patient_doctor_df = patient_doctor_df[['PatientID', 'DoctorID']]
patient_doctor_df.to_csv("patient_doctor_3NF.csv", index=False)
patient_doctor_df.head()

,PatientID,DoctorID
0,101,300001
1,102,300002
2,103,300003
3,104,300004
4,105,300005


In [125]:
patient_doctor_df.shape

(55500, 2)

In [126]:
patient_medication_df = df1NF[['PatientID', 'Medication', 'Test Results']].drop_duplicates()
patient_medication_df = patient_medication_df.merge(medication_df, on='Medication', how='left')
patient_medication_df = patient_medication_df[['PatientID', 'MedicationID', 'Test Results']]
patient_medication_df.to_csv("patient_medication_3NF.csv", index=False)
patient_medication_df.head()

,PatientID,MedicationID,Test Results
0,101,M1,Normal
1,102,M2,Inconclusive
2,103,M3,Normal
3,104,M2,Abnormal
4,105,M4,Abnormal


In [127]:
patient_medication_df.shape

(55500, 3)

In [129]:
patient_insurance_df = df1NF[['PatientID', 'Insurance Provider']].drop_duplicates()
patient_insurance_df = patient_insurance_df.merge(insurance_df, on='Insurance Provider', how='left')
patient_insurance_df = patient_insurance_df[['PatientID', 'InsuranceID']]
patient_insurance_df.to_csv("patient_insurance_3NF.csv", index=False)

In [130]:
patient_insurance_df.head()

,PatientID,InsuranceID
0,101,I1
1,102,I2
2,103,I3
3,104,I2
4,105,I3
